Import Necessary Libraries

In [4]:
import torch
import torch.nn as nn
from collections import defaultdict
import pandas as pd


Build Vocabulary

In [5]:
# Function to build the vocabulary
def build_vocab(file_path):
    # Same as in train.ipynb
    input_vocab = defaultdict(lambda: len(input_vocab))
    output_vocab = defaultdict(lambda: len(output_vocab))

    input_vocab['<PAD>'] = 0
    input_vocab['<UNK>'] = 1
    output_vocab['<PAD>'] = 0
    output_vocab['<UNK>'] = 1
    output_vocab['<SOS>'] = 2
    output_vocab['<EOS>'] = 3

    with open(file_path, 'r') as file:
        for line in file:
            input_sent, output_sent = line.strip().split(',')
            input_sent = input_sent.strip().strip("'")
            output_sent = output_sent.strip().strip("'")

            for char in input_sent:
                input_vocab[char]
            for char in output_sent:
                output_vocab[char]

    input_vocab = dict(input_vocab)
    output_vocab = dict(output_vocab)
    input_vocab_inv = {v: k for k, v in input_vocab.items()}
    output_vocab_inv = {v: k for k, v in output_vocab.items()}

    return input_vocab, output_vocab, input_vocab_inv, output_vocab_inv

# Build vocabularies
input_vocab, output_vocab, input_vocab_inv, output_vocab_inv = build_vocab('/content/Assignment2_train.txt')


Define the Model (Same as in train.ipynb)

In [6]:
class Encoder(nn.Module):
    def __init__(self, input_vocab_size, emb_dim, enc_hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_vocab_size, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True, batch_first=True)

    def forward(self, x):
        embedded = self.embedding(x)
        outputs, hidden = self.rnn(embedded)
        hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        return outputs, hidden

class BahdanauAttention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        self.W_a = nn.Linear(dec_hid_dim, dec_hid_dim)
        self.U_a = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        projected_hidden = self.W_a(hidden.unsqueeze(1))
        energy = torch.tanh(projected_hidden + self.U_a(encoder_outputs))
        attention = self.v(energy).squeeze(2)
        attention_weights = torch.softmax(attention, dim=1)
        return attention_weights

class Decoder(nn.Module):
    def __init__(self, output_vocab_size, emb_dim, enc_hid_dim, dec_hid_dim):
        super().__init__()
        self.output_vocab_size = output_vocab_size
        self.attention = BahdanauAttention(enc_hid_dim, dec_hid_dim)
        self.embedding = nn.Embedding(output_vocab_size, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim, batch_first=True)
        self.fc = nn.Linear(dec_hid_dim, output_vocab_size)

    def forward(self, x, hidden, encoder_outputs):
        x = x.unsqueeze(1)
        embedded = self.embedding(x)
        attention_weights = self.attention(hidden, encoder_outputs)
        attention_weights = attention_weights.unsqueeze(1)
        context_vector = torch.bmm(attention_weights, encoder_outputs)
        rnn_input = torch.cat((embedded, context_vector), dim=2)
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        prediction = self.fc(output.squeeze(1))
        return prediction, hidden.squeeze(0), attention_weights.squeeze(1)

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, input, target, teacher_forcing_ratio=0.5):
        batch_size = input.shape[0]
        target_len = target.shape[1]
        target_vocab_size = self.decoder.output_vocab_size

        outputs = torch.zeros(batch_size, target_len - 1, target_vocab_size).to(self.device)
        attention_scores = torch.zeros(batch_size, target_len - 1, input.shape[1]).to(self.device)

        encoder_outputs, hidden = self.encoder(input)
        x = target[:, 0]  # <SOS> token

        for t in range(1, target_len):
            output, hidden, attention_weights = self.decoder(x, hidden, encoder_outputs)
            attention_scores[:, t - 1] = attention_weights
            outputs[:, t - 1] = output
            teacher_force = torch.rand(1) < teacher_forcing_ratio
            top1 = output.argmax(1)
            x = target[:, t] if teacher_force else top1

        return outputs, attention_scores

# Set model parameters
input_vocab_size = len(input_vocab)
output_vocab_size = len(output_vocab)
embedding_size = 128
enc_hidden_size = 128
dec_hidden_size = 2 * 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model
encoder = Encoder(input_vocab_size, embedding_size, enc_hidden_size)
decoder = Decoder(output_vocab_size, embedding_size, enc_hidden_size, dec_hidden_size)
model = Seq2Seq(encoder, decoder, device).to(device)

Load the Trained Model

In [7]:
model.load_state_dict(torch.load('/content/model.pth', map_location=device))
model.eval()

<ipython-input-7-6f57abfef6f9>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/model.pth', map_location=device))


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(36, 128)
    (rnn): GRU(128, 128, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (attention): BahdanauAttention(
      (W_a): Linear(in_features=256, out_features=256, bias=True)
      (U_a): Linear(in_features=256, out_features=256, bias=True)
      (v): Linear(in_features=256, out_features=1, bias=False)
    )
    (embedding): Embedding(15, 128)
    (rnn): GRU(384, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=15, bias=True)
  )
)

Define Prediction Function

In [8]:
def predict(model, src, src_vocab, tgt_vocab, tgt_inv_vocab, max_len, device):
    src_seq = [src_vocab.get(char, src_vocab['<UNK>']) for char in src]
    src_tensor = torch.tensor(src_seq).unsqueeze(0).to(device)
    tgt_seq = [tgt_vocab['<SOS>']] + [tgt_vocab['<PAD>']] * max_len + [tgt_vocab['<EOS>']]
    tgt_tensor = torch.tensor(tgt_seq).unsqueeze(0).to(device)
    outputs, _ = model(src_tensor, tgt_tensor, 0)
    outputs = outputs.squeeze(0)
    decoder_outputs = []
    for output in outputs:
        output_token = output.argmax(0).item()
        if output_token == tgt_vocab['<EOS>']:
            break
        decoder_outputs.append(tgt_inv_vocab[output_token])
    return "".join(decoder_outputs)


Load Test Dataset and Make Predictions

In [9]:
test_data = []
with open('/content/Assignment2_validation.txt', 'r') as file:
    for line in file:
        input_sent = line.strip().strip("'")
        test_data.append(input_sent)

# Make predictions
max_output_len = 10
predicted_outputs = []
for input_sent in test_data:
    output = predict(model, input_sent, input_vocab, output_vocab, output_vocab_inv, max_output_len, device)
    predicted_outputs.append(output)


Save Output in Excel Format

In [10]:
# Save predictions to Excel
df = pd.DataFrame({'Input': test_data, 'Predicted Output': predicted_outputs})
df.to_excel('predictions.xlsx', index=False)
print("Predictions saved to 'predictions.xlsx'")

Predictions saved to 'predictions.xlsx'
